In [2]:
import pandas as pd

#Import data
HDNames= ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','hal','HeartDisease']
Data = pd.read_excel('Ch3.ClevelandData.xlsx', names=HDNames)


print(Data.head(20))
print(Data.info())
summary = Data.describe()
print(summary)


#Removing missing values
import numpy as np

DataNew = Data.replace('?', np.nan)

print(DataNew.info())

print(DataNew.describe())


print(DataNew.isnull().sum())

DataNew = DataNew.dropna()

print(DataNew.info())

print(DataNew.isnull().sum())



#Divide DataFrame
InputNames = HDNames
InputNames.pop()
Input = pd.DataFrame(DataNew.iloc[:, 0:13],columns=InputNames)

Target = pd.DataFrame(DataNew.iloc[:, 13],columns=['HeartDisease'])

#Data scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print(scaler.fit(Input))
InputScaled = scaler.fit_transform(Input)


InputScaled = pd.DataFrame(InputScaled,columns=InputNames)

summary = InputScaled.describe()
summary = summary.transpose()
print(summary)

#Data visualitation
#DataScaled = pd.concat([InputScaled, Target], axis=1)

import matplotlib.pyplot as plt
boxplot = InputScaled.boxplot(column=InputNames,showmeans=True)
plt.show()

pd.plotting.scatter_matrix(InputScaled, figsize=(6, 6))
plt.show()

CorData = InputScaled.corr(method='pearson')

with pd.option_context('display.max_rows', None, 'display.max_columns', CorData.shape[1]):
    print(CorData)

plt.matshow(CorData)
plt.xticks(range(len(CorData.columns)), CorData.columns)
plt.yticks(range(len(CorData.columns)), CorData.columns)
plt.colorbar()
plt.show()

#Split the data
from sklearn.model_selection import train_test_split

Input_train, Input_test, Target_train, Target_test = train_test_split(InputScaled, Target, test_size = 0.30, random_state = 5)
print(Input_train.shape)
print(Input_test.shape)
print(Target_train.shape)
print(Target_test.shape)

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(30, input_dim=13, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(Input_train, Target_train, epochs=1000, verbose=1)

model.summary()

score = model.evaluate(Input_test, Target_test, verbose=0)

print('Keras Model Accuracy = ',score[1])

Target_Classification = model.predict(Input_test)
Target_Classification = (Target_Classification > 0.5)



from sklearn.metrics import confusion_matrix

print(confusion_matrix(Target_test, Target_Classification))

    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0    67    1   4       160   286    0        2      108      1      1.5   
1    67    1   4       120   229    0        2      129      1      2.6   
2    37    1   3       130   250    0        0      187      0      3.5   
3    41    0   2       130   204    0        2      172      0      1.4   
4    56    1   2       120   236    0        0      178      0      0.8   
5    62    0   4       140   268    0        2      160      0      3.6   
6    57    0   4       120   354    0        0      163      1      0.6   
7    63    1   4       130   254    0        2      147      0      1.4   
8    53    1   4       140   203    1        2      155      1      3.1   
9    57    1   4       140   192    0        0      148      0      0.4   
10   56    0   2       140   294    0        2      153      0      1.3   
11   56    1   3       130   256    1        2      142      1      0.6   
12   44    1   2       12

/home/hygjob/testbed/datascience/ve/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/hygjob/testbed/datascience/ve/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/hygjob/testbed/datascience/ve/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


StandardScaler(copy=True, with_mean=True, with_std=True)
          count          mean       std       min       25%       50%  \
age       296.0 -2.550512e-16  1.001693 -2.823438 -0.720814  0.164501   
sex       296.0  6.001206e-18  1.001693 -1.443376 -1.443376  0.692820   
cp        296.0  1.860374e-16  1.001693 -2.263690 -0.173043 -0.173043   
trestbps  296.0  1.920386e-16  1.001693 -2.121540 -0.656413 -0.092903   
chol      296.0  6.601326e-17  1.001693 -2.334999 -0.700097 -0.084604   
fbs       296.0  2.700542e-17  1.001693 -0.406638 -0.406638 -0.406638   
restecg   296.0 -4.800964e-17  1.001693 -1.000046 -1.000046  0.006803   
thalach   296.0  2.160434e-16  1.001693 -3.426010 -0.723490  0.148291   
exang     296.0  1.342770e-16  1.001693 -0.698167 -0.698167 -0.698167   
oldpeak   296.0  6.001206e-17  1.001693 -0.903324 -0.903324 -0.215962   
slope     296.0 -6.901386e-17  1.001693 -0.975788 -0.975788  0.656038   
ca        296.0  1.200241e-17  1.001693 -0.723832 -0.723832 -0.7238

<Figure size 640x480 with 1 Axes>

<Figure size 600x600 with 169 Axes>

...

207/207 [==============================] - 0s 98us/step - loss: 8.9507e-04 - acc: 1.0000
Epoch 650/1000
207/207 [==============================] - 0s 99us/step - loss: 8.9228e-04 - acc: 1.0000
Epoch 651/1000
207/207 [==============================] - 0s 103us/step - loss: 8.8589e-04 - acc: 1.0000
Epoch 652/1000
207/207 [==============================] - 0s 102us/step - loss: 8.7931e-04 - acc: 1.0000
Epoch 653/1000
207/207 [==============================] - 0s 99us/step - loss: 8.7771e-04 - acc: 1.0000
Epoch 654/1000
207/207 [==============================] - 0s 104us/step - loss: 8.7288e-04 - acc: 1.0000
Epoch 655/1000
207/207 [==============================] - 0s 102us/step - loss: 8.6559e-04 - acc: 1.0000
Epoch 656/1000
207/207 [==============================] - 0s 104us/step - loss: 8.6107e-04 - acc: 1.0000
Epoch 657/1000
207/207 [==============================] - 0s 100us/step - loss: 8.5537e-04 - acc: 1.0000
Epoch 658/1000
207/207 [==============================] - 0s 101us/step -

207/207 [==============================] - 0s 102us/step - loss: 5.7290e-04 - acc: 1.0000
Epoch 727/1000
207/207 [==============================] - 0s 102us/step - loss: 5.7183e-04 - acc: 1.0000
Epoch 728/1000
207/207 [==============================] - 0s 103us/step - loss: 5.6685e-04 - acc: 1.0000
Epoch 729/1000
207/207 [==============================] - 0s 105us/step - loss: 5.6293e-04 - acc: 1.0000
Epoch 730/1000
207/207 [==============================] - 0s 98us/step - loss: 5.5919e-04 - acc: 1.0000
Epoch 731/1000
207/207 [==============================] - 0s 106us/step - loss: 5.5581e-04 - acc: 1.0000
Epoch 732/1000
207/207 [==============================] - 0s 104us/step - loss: 5.5743e-04 - acc: 1.0000
Epoch 733/1000
207/207 [==============================] - 0s 106us/step - loss: 5.5281e-04 - acc: 1.0000
Epoch 734/1000
207/207 [==============================] - 0s 105us/step - loss: 5.4929e-04 - acc: 1.0000
Epoch 735/1000
207/207 [==============================] - 0s 117us/step

207/207 [==============================] - 0s 101us/step - loss: 3.7800e-04 - acc: 1.0000
Epoch 804/1000
207/207 [==============================] - 0s 95us/step - loss: 3.7495e-04 - acc: 1.0000
Epoch 805/1000
207/207 [==============================] - 0s 103us/step - loss: 3.7281e-04 - acc: 1.0000
Epoch 806/1000
207/207 [==============================] - 0s 100us/step - loss: 3.7152e-04 - acc: 1.0000
Epoch 807/1000
207/207 [==============================] - 0s 101us/step - loss: 3.6729e-04 - acc: 1.0000
Epoch 808/1000
207/207 [==============================] - 0s 97us/step - loss: 3.6616e-04 - acc: 1.0000
Epoch 809/1000
207/207 [==============================] - 0s 104us/step - loss: 3.6390e-04 - acc: 1.0000
Epoch 810/1000
207/207 [==============================] - 0s 103us/step - loss: 3.6199e-04 - acc: 1.0000
Epoch 811/1000
207/207 [==============================] - 0s 102us/step - loss: 3.6034e-04 - acc: 1.0000
Epoch 812/1000
207/207 [==============================] - 0s 103us/step 

207/207 [==============================] - 0s 100us/step - loss: 2.5132e-04 - acc: 1.0000
Epoch 881/1000
207/207 [==============================] - 0s 105us/step - loss: 2.4952e-04 - acc: 1.0000
Epoch 882/1000
207/207 [==============================] - 0s 101us/step - loss: 2.4903e-04 - acc: 1.0000
Epoch 883/1000
207/207 [==============================] - 0s 98us/step - loss: 2.4880e-04 - acc: 1.0000
Epoch 884/1000
207/207 [==============================] - 0s 100us/step - loss: 2.4625e-04 - acc: 1.0000
Epoch 885/1000
207/207 [==============================] - 0s 101us/step - loss: 2.4514e-04 - acc: 1.0000
Epoch 886/1000
207/207 [==============================] - 0s 105us/step - loss: 2.4367e-04 - acc: 1.0000
Epoch 887/1000
207/207 [==============================] - 0s 106us/step - loss: 2.4218e-04 - acc: 1.0000
Epoch 888/1000
207/207 [==============================] - 0s 110us/step - loss: 2.4084e-04 - acc: 1.0000
Epoch 889/1000
207/207 [==============================] - 0s 104us/step

207/207 [==============================] - 0s 107us/step - loss: 1.7098e-04 - acc: 1.0000
Epoch 958/1000
207/207 [==============================] - 0s 110us/step - loss: 1.7007e-04 - acc: 1.0000
Epoch 959/1000
207/207 [==============================] - 0s 106us/step - loss: 1.6940e-04 - acc: 1.0000
Epoch 960/1000
207/207 [==============================] - 0s 99us/step - loss: 1.6816e-04 - acc: 1.0000
Epoch 961/1000
207/207 [==============================] - 0s 97us/step - loss: 1.6812e-04 - acc: 1.0000
Epoch 962/1000
207/207 [==============================] - 0s 97us/step - loss: 1.6678e-04 - acc: 1.0000
Epoch 963/1000
207/207 [==============================] - 0s 102us/step - loss: 1.6620e-04 - acc: 1.0000
Epoch 964/1000
207/207 [==============================] - 0s 105us/step - loss: 1.6493e-04 - acc: 1.0000
Epoch 965/1000
207/207 [==============================] - 0s 100us/step - loss: 1.6470e-04 - acc: 1.0000
Epoch 966/1000
207/207 [==============================] - 0s 100us/step -